# MBA Investment Management – Performance Evaluation Demo

In this notebook, we will walk through:
1. Downloading stock data and Fama-French factor data
2. Constructing a simple portfolio
3. Running regressions to get CAPM and multi-factor alphas
4. Calculating risk-adjusted performance metrics (Sharpe, Information, Treynor)

This follows from the lecture slides on performance evaluation and uses real market data.


## 1. Imports and Helper Functions
We will need a few common libraries: `yfinance`, `pandas_datareader`, `pandas`, `numpy`, `matplotlib`, `statsmodels`.

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
from pandas_datareader import data as web
import statsmodels.api as sm

# Make plots appear inline (in a Jupyter environment)
%matplotlib inline

def get_stock_data(tickers, start_date, end_date):
    """
    Download historical stock data (Close prices) for a list of tickers.
    Returns daily returns.
    """
    data = yf.download(tickers, start=start_date, end=end_date)["Close"]
    returns = data.pct_change().dropna()
    return returns


## 2. Data Preparation

### 2.1 Download Stock and Market Data
Let's get 5 years of daily data for several stocks plus the S&P 500 index (to use as our market portfolio).

In [4]:
# Define date range - ~5 years of data
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=5*365)

# Define tickers - some popular stocks
market_ticker = '^GSPC'  # S&P 500
stock_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA', 'JPM', 'V', 'WMT']

# Combined list with market first (useful for certain calculations)
tickers = [market_ticker] + stock_tickers

# Download the data
returns = get_stock_data(tickers, start_date, end_date)

returns.head()

[*********************100%***********************]  11 of 11 completed


Ticker,AAPL,AMZN,GOOGL,JPM,META,MSFT,NVDA,TSLA,V,WMT,^GSPC
Date,,,,,,,,,,,
2020-04-03,-0.014371,-0.006379,-0.021781,-0.029558,-0.025349,-0.009211,-0.045250,0.056197,-0.035199,0.006996,-0.015137
2020-04-06,0.087237,0.047729,0.082813,0.064366,0.073745,0.074368,0.100406,0.075478,0.115838,0.055155,0.070331
2020-04-07,-0.011582,0.007013,-0.000532,0.013190,0.019813,-0.010770,-0.034911,0.056582,-0.005016,-0.032363,-0.001603
2020-04-08,0.025595,0.015609,0.020667,0.040380,0.032281,0.010031,0.030576,0.006215,0.037665,-0.001230,0.034056
2020-04-09,0.007216,-0.000117,-0.000356,0.089713,0.005222,0.000061,-0.014984,0.044020,-0.007145,-0.000328,0.014487


### 2.2 Calculate Excess Returns
We assume a constant risk-free rate (annualized 3%) and convert it to a daily rate for simplicity. Then we subtract it from the raw returns to get excess returns.

In [5]:
risk_free_rate_annual = 0.03  # 3% annual
risk_free_rate_daily = risk_free_rate_annual / 252  # approximate daily risk-free

excess_returns = returns.subtract(risk_free_rate_daily, axis=0)
excess_returns.head()

Ticker,AAPL,AMZN,GOOGL,JPM,META,MSFT,NVDA,TSLA,V,WMT,^GSPC
Date,,,,,,,,,,,
2020-04-03,-0.014490,-0.006498,-0.021900,-0.029677,-0.025468,-0.009330,-0.045369,0.056078,-0.035318,0.006877,-0.015256
2020-04-06,0.087118,0.047610,0.082694,0.064247,0.073626,0.074249,0.100287,0.075359,0.115719,0.055036,0.070212
2020-04-07,-0.011701,0.006894,-0.000651,0.013071,0.019694,-0.010889,-0.035030,0.056463,-0.005135,-0.032482,-0.001722
2020-04-08,0.025476,0.015490,0.020548,0.040260,0.032162,0.009912,0.030456,0.006096,0.037546,-0.001349,0.033937
2020-04-09,0.007097,-0.000237,-0.000475,0.089594,0.005103,-0.000058,-0.015103,0.043901,-0.007264,-0.000447,0.014368


### 2.3 Fama-French Data
We'll grab the Fama-French 3-Factor data from Kenneth French’s library using `pandas_datareader`. Note that 'F-F_Research_Data_Factors' is provided in monthly frequency. If you want daily factor data, you need a different dataset from the library. For this example, let's proceed with the monthly data. We'll also convert the daily stock returns to monthly to match.


In [6]:
# Get monthly Fama-French 3-factor data (Mkt-RF, SMB, HML, RF)
ff_factors = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start='1950')[0]
# Convert to decimals
ff_factors = ff_factors / 100
ff_factors.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1950-01,0.0170,0.0333,0.0012,0.0009
1950-02,0.0148,0.0003,-0.0079,0.0009
1950-03,0.0126,-0.0144,-0.0281,0.0010
1950-04,0.0394,0.0194,0.0130,0.0009
1950-05,0.0431,-0.0210,0.0046,0.0010


### 2.4 Aligning Frequencies (Converting to Monthly)
Because our stock returns are currently daily, let's convert them to monthly (taking the arithmetic sum of daily returns for the month is usually done by `(1 + daily_returns).prod() - 1` for total return over the month). Then we can easily join them with the Fama-French monthly data.


In [9]:
# Convert daily returns to monthly returns
monthly_returns = (1 + returns).resample('M').prod() - 1
monthly_excess_returns = monthly_returns.subtract(risk_free_rate_annual/12, axis=1)

# We'll rename columns so that the market ticker is 'Market'
rename_map = {market_ticker: 'Market'}
monthly_excess_returns = monthly_excess_returns.rename(columns=rename_map)

# Join with Fama-French
# Ensure consistent alignment by converting both indices to end-of-month timestamps.

ff_data = ff_factors.copy()
# Convert Fama-French index to end-of-month timestamps
ff_data.index = ff_data.index.to_timestamp('M')

# Convert monthly_excess_returns index to end-of-month timestamps (if not already)
monthly_excess_returns.index = monthly_excess_returns.index.to_period('M').to_timestamp('M')

# Perform the merge
combined_data = pd.merge(monthly_excess_returns, ff_data, left_index=True, right_index=True, how='inner')

combined_data.head()

,AAPL,AMZN,GOOGL,JPM,META,MSFT,NVDA,TSLA,V,WMT,Market,Mkt-RF,SMB,HML,RF
Date,,,,,,,,,,,,,,,
2020-04-30,0.197027,0.286827,0.203108,0.103146,0.291577,0.151757,0.141587,0.717922,0.133023,0.021942,0.150070,0.1365,0.0245,-0.0134,0.0000
2020-05-31,0.082594,-0.015285,0.061969,0.013686,0.097056,0.022891,0.212157,0.065439,0.091765,0.022640,0.042782,0.0558,0.0249,-0.0485,0.0001
2020-06-30,0.144886,0.127067,-0.013292,-0.035898,0.006296,0.108060,0.068097,0.290686,-0.013102,-0.036999,0.015888,0.0246,0.0269,-0.0223,0.0001
2020-07-31,0.162632,0.144614,0.046793,0.034941,0.114644,0.004870,0.115107,0.322511,-0.016840,0.077814,0.052601,0.0577,-0.0230,-0.0144,0.0001
2020-08-31,0.214069,0.087961,0.092651,0.034234,0.153332,0.100252,0.257492,0.738952,0.112576,0.074941,0.067565,0.0763,-0.0028,-0.0288,0.0001


## 3. Construct a Portfolio
For simplicity, let's assume an equally weighted portfolio of our 10 stocks (excluding the market index). We'll compute its monthly returns and then proceed to measure performance.


In [10]:
# List of actual stocks (exclude 'Market' which was ^GSPC)
portfolio_stocks = [t for t in monthly_excess_returns.columns if t != 'Market']

# Equally weighted portfolio
weights = np.array([1/len(portfolio_stocks)]*len(portfolio_stocks))

# Calculate portfolio returns (just do a dot product across columns)
portfolio_returns = monthly_excess_returns[portfolio_stocks].mul(weights).sum(axis=1)
portfolio_returns.name = 'Portfolio'

portfolio_returns.head()

Date
2020-04-30    0.224791
2020-05-31    0.065491
2020-06-30    0.064580
2020-07-31    0.100709
2020-08-31    0.186646
Freq: ME, Name: Portfolio, dtype: float64

## 4. Factor Regressions and Alpha
We'll run a CAPM regression, then a 3-factor (Fama-French) regression, on our portfolio.

### 4.1 CAPM Regression
CAPM: 
\[
r_{p,t} - r_{f,t} = \alpha + \beta (r_{m,t} - r_{f,t}) + \epsilon_t
\]

In the Fama-French data, `Mkt-RF` is already `(Market - RF)`. If we want to do a basic CAPM in that style, we can just do:
\[
r_{p,t} - r_{f,t} = \alpha + \beta (\text{Mkt-RF}) + \epsilon_t
\]
But note that our `portfolio_returns` are already *excess returns*.

In [11]:
# Merge portfolio returns with factors
reg_data = pd.concat([
    portfolio_returns,
    ff_data['Mkt-RF']
], axis=1).dropna()

Y = reg_data['Portfolio']  # portfolio excess return
X = reg_data['Mkt-RF']
X = sm.add_constant(X)

capm_model = sm.OLS(Y, X).fit()
print(capm_model.summary())

                            OLS Regression Results                            
Dep. Variable:              Portfolio   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.793
Method:                 Least Squares   F-statistic:                     215.5
Date:                Tue, 01 Apr 2025   Prob (F-statistic):           1.13e-20
Time:                        12:34:28   Log-Likelihood:                 113.93
No. Observations:                  57   AIC:                            -223.9
Df Residuals:                      55   BIC:                            -219.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0095      0.005      2.067      0.0

### 4.2 Three-Factor Regression
Now we extend to the Fama-French 3-factor model:
\[
r_{p,t} - r_{f,t} = \alpha + \beta (\text{Mkt-RF}) + s\,\text{SMB} + h\,\text{HML} + \epsilon_t
\]


In [12]:
# Merge the portfolio returns with Mkt-RF, SMB, HML
ff_3factor_data = pd.concat([
    portfolio_returns,
    ff_data[['Mkt-RF','SMB','HML']]
], axis=1).dropna()

Y = ff_3factor_data['Portfolio']
X = ff_3factor_data[['Mkt-RF','SMB','HML']]
X = sm.add_constant(X)

ff3_model = sm.OLS(Y, X).fit()
print(ff3_model.summary())

                            OLS Regression Results                            
Dep. Variable:              Portfolio   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     128.0
Date:                Tue, 01 Apr 2025   Prob (F-statistic):           2.93e-24
Time:                        12:34:35   Log-Likelihood:                 128.65
No. Observations:                  57   AIC:                            -249.3
Df Residuals:                      53   BIC:                            -241.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0121      0.004      3.292      0.0

## 5. Risk-Adjusted Performance Measures
We’ll compute:
- **Sharpe Ratio**
- **Information Ratio** (relative to CAPM)
- **Treynor Ratio**

### 5.1 Sharpe Ratio
Recall:
\[ \text{Sharpe Ratio} = \frac{\bar{r}_p - r_f}{\sigma_p} \]
Since we’ve already been working in excess returns, we can just do:
\[ \text{Sharpe Ratio} = \frac{\bar{r}_p}{\sigma_p} \]
where \( \bar{r}_p\) is the average excess return on the portfolio (monthly if that’s how your data is). We'll then annualize that if needed.


In [19]:
# Monthly average (excess) return of portfolio
mean_rp_monthly = portfolio_returns.mean()
# Monthly std dev
std_rp_monthly = portfolio_returns.std()

sharpe_monthly = mean_rp_monthly / std_rp_monthly

# If you want an annualized Sharpe ratio for monthly data:
# annualized_sharpe = sharpe_monthly * np.sqrt(12)

print("Monthly Sharpe Ratio = ", round(sharpe_monthly, 4))
print("Annualized Sharpe Ratio = ", round(sharpe_monthly * np.sqrt(12), 4))

Monthly Sharpe Ratio =  0.3453
Annualized Sharpe Ratio =  1.1961


### 5.2 Information Ratio
The Information Ratio is defined as:
\[ IR = \frac{\alpha_p}{\sigma(\epsilon_p)} \]
where \( \alpha_p \) is the portfolio’s alpha (e.g., from CAPM) and \( \sigma(\epsilon_p) \) is the standard deviation of the residual.

From our CAPM regression, we can extract alpha and the residuals (i.e., fitted errors).

In [15]:
# From our CAPM regression:
alpha_capm = capm_model.params["const"]
residuals = capm_model.resid
sigma_e = residuals.std()
# From our FF3 regression:
alpha_ff3 = ff3_model.params["const"]
residuals_ff3 = ff3_model.resid
sigma_e_ff3 = residuals_ff3.std()

# IR in monthly terms
IR_monthly = alpha_capm / sigma_e
IR_monthly_ff3 = alpha_ff3 / sigma_e_ff3


# If you want to interpret alpha as an annual figure, you might multiply alpha by 12
# but should also scale the std dev consistently. We'll just keep everything in monthly.

print("CAPM")
print("CAPM alpha (monthly) =", round(alpha_capm, 5))
print("Residual stdev (monthly) =", round(sigma_e, 5))
print("Information Ratio (monthly) =", round(IR_monthly, 4))

print("FF3")
print("FF3 alpha (monthly) =", round(alpha_ff3, 5))
print("Residual stdev (monthly) =", round(sigma_e_ff3, 5))
print("Information Ratio (monthly) =", round(IR_monthly_ff3, 4))

CAPM
CAPM alpha (monthly) = 0.00953
Residual stdev (monthly) = 0.03308
Information Ratio (monthly) = 0.2882
FF3
FF3 alpha (monthly) = 0.01211
Residual stdev (monthly) = 0.02555
Information Ratio (monthly) = 0.474


### 5.3 Treynor Ratio
The Treynor ratio uses *beta* rather than total volatility:
\[ T = \frac{\bar{r}_p - r_f}{\beta_p}. \]
Again, we’re working in excess returns for the portfolio, so it is simply:
\[ T = \frac{\bar{r}_p}{\beta_p}. \]

In [18]:
# CAPM beta is in capm_model.params["Mkt-RF"]
beta_capm = capm_model.params["Mkt-RF"]

treynor_monthly = mean_rp_monthly / beta_capm

# If needed, you could annualize the numerator. We'll just keep monthly.

print("CAPM Beta =", round(beta_capm, 4))
print("Market Risk Premium (monthly) =", round(mean_rp_monthly, 5))
print("Treynor Ratio (monthly) =", round(treynor_monthly, 5))

CAPM Beta = 1.2884
Market Risk Premium (monthly) = 0.02534
Treynor Ratio (monthly) = 0.01967


#  Brimson's Decomposition

In [20]:



def brinson_decomposition(df):
    """
    Given a DataFrame df with columns:
        'w_p': portfolio weight in each asset class
        'r_p': portfolio return in each asset class
        'w_b': benchmark weight in each asset class
        'r_b': benchmark return in each asset class
    this function computes and returns the Brinson decomposition:
      1) Asset Allocation
      2) Selection
      3) Interaction
    plus the total difference in returns.
    """
    # Total portfolio and benchmark returns
    r_p = (df['w_p'] * df['r_p']).sum()
    r_b = (df['w_b'] * df['r_b']).sum()

    # Decomposition components
    asset_allocation = ((df['w_p'] - df['w_b']) * df['r_b']).sum()
    selection = (df['w_b'] * (df['r_p'] - df['r_b'])).sum()
    interaction = ((df['w_p'] - df['w_b']) * (df['r_p'] - df['r_b'])).sum()

    # Total difference in returns
    total_diff = r_p - r_b

    return {
        'Portfolio Return': r_p,
        'Benchmark Return': r_b,
        'Asset Allocation': asset_allocation,
        'Selection': selection,
        'Interaction': interaction,
        'Total Difference': total_diff
    }


# EXAMPLE USAGE:
# -------------------------------------------------
# Imagine we have a small universe of 3 “asset classes” (or sectors).
# Create a DataFrame with columns for portfolio weights/returns
# and benchmark weights/returns:

data = {
    'AssetClass':    ['Equity', 'Bond', 'RealEstate'],
    'w_p':           [0.60,      0.30,   0.10],   # Portfolio weights
    'r_p':           [0.10,      0.02,   0.08],   # Portfolio returns
    'w_b':           [0.50,      0.40,   0.10],   # Benchmark weights
    'r_b':           [0.08,      0.03,   0.07]    # Benchmark returns
}

df_brinson = pd.DataFrame(data).set_index('AssetClass')

# Perform Brinson decomposition
results = brinson_decomposition(df_brinson)

print("Brinson Decomposition Results:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")

Brinson Decomposition Results:
Portfolio Return: 0.0740
Benchmark Return: 0.0590
Asset Allocation: 0.0050
Selection: 0.0070
Interaction: 0.0030
Total Difference: 0.0150


### Conclusion
In this notebook, we've shown how to:
- Pull real data from Yahoo Finance and Fama-French,
- Construct an equal-weight portfolio,
- Run factor regressions (CAPM, Fama-French 3-factor),
- Compute common risk-adjusted performance metrics (Sharpe, Information, Treynor).

This provides a foundation for further exploration and experimentation in an MBA-level investment management course.